In [31]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors as mcolors

%matplotlib inline
import seaborn as sns
sns.set_style("white")
import plotly
import plotly.graph_objs as go

import webcolors
from colorharmonies import Color, complementaryColor, triadicColor, splitComplementaryColor, tetradicColor, analogousColor, monochromaticColor

import cv2 #openCV library

from sklearn.cluster import KMeans
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.neighbors import NearestNeighbors

#pd.set_option("display.max_rows", None)

# Add - plt.axis('off') - for photo plotting

# functions

### import photo OpenCV

In [14]:
def import_photo(name:str): #defines what type to input
    interior_photo_1 = cv2.imread(f"images/{name}.jpg")
    interior_photo_1 = cv2.cvtColor(interior_photo_1, cv2.COLOR_BGR2RGB)
    return interior_photo_1 #the 3d nparray

### plot photo (show photo)

In [ ]:
# can call show_photo(import_photo) to show photo imported in def import_photo

def show_photo(import_photo):
    plt.axis('off')
    plt.imshow(interior_photo_1)

In [ ]:
# interior_photo_2d = interior_photo_1.reshape((-1,3)) #transform into 2d nparry

### elbow plot to check optimal amount of clusters

In [2]:
# interior_photo_1_df = pd.DataFrame(data=interior_photo_2d, columns=["Red", "Green", "Blue"]) --SAME as X
# X = interior_photo_1_df[["Red", "Green", "Blue"]] - the RGB values of the imported photo

def plot_elbow(X):
    inertias = {}   
    for k in range(1,15):
        cl = KMeans(n_clusters=k).fit(X)
        inertias[k] = cl.inertia_
    plt.figure(figsize=(10,6))
    sns.lineplot(x=list(inertias.keys()),
                 y=list(inertias.values()));
    plt.title("Optimal number of clusters");

### Kmeans with OpenCV and define amount of clusters + show clusters in photo

In [ ]:
def clusters_photo(name:str, K=5): #input "name of photo" and amount of clusters
    interior_photo_1 = import_photo(name) #call import_photo function
    pixels = interior_photo_1.reshape((-1,3))

    # convert to np.float32
    pixels = np.float32(pixels)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    #K = 5 
    ret,label,center=cv2.kmeans(pixels,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res = res.reshape((interior_photo_1.shape))
    
    plt.axis('off')
    plt.imshow(res)
    plt.show()

### prepare photo data for plotting

In [30]:
def df_preparation(name, K=5):
    
    open_image = import_photo(name)

    #convert from 3darry to 2darry   
    interior_photo_2d = open_image.reshape((-1,3)) 

    #create DF with RGB columns
    interior_photo_1_df = pd.DataFrame(data=interior_photo_2d, columns=["Red", "Green", "Blue"]) #create dataframe

    #fit kmeans model
    X = interior_photo_1_df[["Red", "Green", "Blue"]] ###SAME AS interior_photo_1_df
    clusters = KMeans(K) #have to define the amount of clusters already before
    clusters.fit(X)
    
    #add fitted clusters as new column to DF
    X_clusters = X.assign(cluster=clusters.predict(X)) #add column with the clusters

    return X_clusters

### plot clusters in 3d

In [3]:
def clusters_3d(name, K): #able to plot on prepared DF (+ input amount of clusters)
    X_clusters = df_preparation(name, K)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    r = np.array(X_clusters["Red"])
    g = np.array(X_clusters["Green"])
    b = np.array(X_clusters["Blue"])
    
    ax.scatter(r,g,b, marker="s", c=X_clusters["cluster"], s=40, cmap="RdBu")

    plt.show()

    print(f"Inertia: {clusters.inertia_}")

### create cluster labels - not working yet

In [ ]:
#to do

def cluster_labels():
    X = interior_photo_1_df[["Red", "Green", "Blue"]]
    clusters = KMeans(n_clusters=5)
    clusters.fit(X)
    return clusters.labels_




# says it wont print it because too much data and might crash
# however before it was returning: [1 1 1 ... 0 0 0]

X = interior_photo_1_df[["Red", "Green", "Blue"]]
clusters = KMeans(n_clusters=5)
clusters.fit(X)
labels = clusters.labels_
labels = list(labels)
print(labels)
----------------

# returns [0 0 0 ... 4 4 4]

X = interior_photo_1_df[["Red", "Green", "Blue"]]
clusters = KMeans(n_clusters=5)
clusters.fit(X)
print(clusters.labels_)
----------------

# are they the same?
# list(clusters.labels_) == list(labels) TRUE
# list(clusters.labels_) != list(labels) FALSE

### get centroids - not working yet

In [ ]:
#to do
# 5 clusters, 5 lists of centroids

centroids = clusters.cluster_centers_
print(centroids)

def get_centroids(): #not sure what to input
    centroids = clusters.cluster_centers_
    return centroids

### percentages - not working yet

In [ ]:
def get_percentage(centroids): 
    percent=[]
    for i in range(len(centroids)):
        labels = list(labels)
        j=labels.count(i)
        j=j/(len(labels))
        percent.append(j)
    return percent

#error: local variable 'labels' referenced before assignment

### pie chart

In [ ]:
def plot_pie(centroids):
    plt.pie(percent,colors=np.array(centroids/255),labels=np.arange(len(centroids)))
    plt.show()

### RGB to HEX - is this necessary?

In [ ]:
from matplotlib.colors import rgb2hex

[rgb2hex((centroids/255)[i,:]) for i in range(len(centroids/255))]

#hex_code = [rgb2hex((centroids/255)[i,:]) for i in range(len(centroids/255))]

#RGBA values should be within 0-1 range : centroids / 255 which is RGB value

#double checking https://www.rapidtables.com/convert/color/hex-to-rgb.html



### Get color name

In [ ]:
#https://stackoverflow.com/questions/9694165/convert-rgb-color-to-english-color-name-like-green-with-python
#https://pypi.org/project/webcolors/1.3/

def closest_colour2(requested_colour): #input RGB tuple. type of webcolors is dict_items()
    min_colours = {}
    for key, name in hex_dict.items(): #hex to rgb
        r, g, b = webcolors.hex_to_rgb(key) #works for any transformation
        red = (r - requested_colour[0]) ** 2 #kind of KNN but manually
        green = (g - requested_colour[1]) ** 2
        blue = (b - requested_colour[2]) ** 2
        min_colours[(red + green + blue)] = name #key rgb, value name
    return min_colours[min(min_colours.keys())] #find minimum distance to color

def get_colour_name2(requested_colour):
    rgb_to_hex = webcolors.rgb_to_hex(requested_colour)
    try:
        closest_name = actual_name = hex_dict[rgb_to_hex] #triyng to find the acutal name in list
    except KeyError:
        closest_name = closest_colour2(requested_colour) #name with minimum distance
        actual_name = None
    return actual_name, closest_name

requested_colour = (33, 171, 205) #from centroids
actual_name, closest_name = get_colour_name2(requested_colour)

print("Actual colour name:", actual_name, ", closest colour name:", closest_name)

### Complimentary color

In [ ]:
#https://github.com/baptistemanteau/colorharmonies

# from library, building blocks:
# 1. input R G B values from centroid 
# 2. return R G B values of opposite
# 3. find color name or closest one
# 4. display color

def complementary_color():
    brown1 = Color([130, 87, 55],"","")
    return complementaryColor(brown1)



